In [1]:
from keras.datasets import boston_housing
from keras import models, layers
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py

In [2]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
train_data.shape

(404, 13)

### Normalize the feature data

In [3]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
train_data = (train_data - mean ) / std

# always use mean and std from TRAIN data
test_data = (test_data - mean) / std
train_data.shape

(404, 13)

### Build model generator

In [4]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(13,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae']) # mean absolute error
    return model

Note that the network ends with a single unit and no activation function, because the layer will be linear. This is because we are solving a regression problem, predicting a singular, continous value. An activation function would squash the range, which we don't want

### K-Fold CV
Because the dataset is small, building a validation set out of the training data would be even smaller, causing a high variance in validation scores. The best practice is to se K-fold CV  
Also, the strategy here is to CV the train data to tune the hyperparameters; then, use the entire training dataset with the tuned hyperparamters for the final model

In [8]:
# although the book rolls their own, we'll use sklearn
K = 3
NUM_EPOCHS = 100

kf = KFold(n_splits=K)
num_val_samples = len(train_data) // K
all_mae = []

for i, (train, test) in enumerate(kf.split(train_data)):
    print('Running Fold', i+1, '/', K)
    model = build_model()
    history = model.fit(train_data[train], train_targets[train],
                        validation_data=(train_data[test], train_targets[test]),
                        epochs=NUM_EPOCHS, batch_size=1, verbose=0)
    mae_hist = history.history['mae'] # results at each epoch
    all_mae.append(mae_hist)
print('\nMean MAE', np.mean(all_mae))

#todo: find mean of each epoch across all folds; plot!

Running Fold 1 / 3
Running Fold 2 / 3
Running Fold 3 / 3

Mean MAE 1.8130395


In [ ]:
# _, mae = model.evaluate(train_data[test], train_targets[test]) 

One thing the book does when rolling their own k-fold CV procedure is use a very small validation set within each fold.
Also, by coding the CV by hand, it's easy to get per epoch MAE, which can be averaged across all folds and graphed

### Notes on batch sizing
The link below discusses batch, stochastic and mini-batch gradient descent
https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/